In [ ]:
from graphviz import Digraph

dot = Digraph(comment='Sample Processing Diagram')
dot.attr(rankdir='TB', nodesep='0.2', ranksep='0.4')

node_style = {'shape': 'rect', 'style': 'filled,rounded', 'fillcolor': '#AED6F1', 'fontname': 'Helvetica', 'fontsize': '10'}
anal_node_style = {'shape': 'rect', 'style': 'filled,rounded', 'fillcolor': 'khaki', 'fontname': 'Helvetica', 'fontsize': '10'}
pre_node_style = {'shape': 'rect', 'style': 'filled,rounded', 'fillcolor': 'lightsalmon', 'fontname': 'Helvetica', 'fontsize': '10'}
edge_style = {'fontsize': '9', 'fontname': 'Helvetica', 'color': '#2E86C1'}
anal_1_edge_style = {'fontsize': '9', 'fontname': 'Helvetica', 'color': 'mediumseagreen'}
anal_2_edge_style = {'fontsize': '9', 'fontname': 'Helvetica', 'color': 'orchid'}
pre_edge_style = {'fontsize': '9', 'fontname': 'Helvetica', 'color': 'salmon'}

# Nodes

dot.node('UC', 'UC', **node_style)
dot.node('UM', 'U-M', **node_style)
dot.node('OSU', 'OSU', **node_style)
dot.node('MUSC', 'MUSC', **node_style)
dot.node('MD', 'MD Anderson', **node_style)
dot.node('UL', 'UofL', **node_style)
dot.node('Pre', 'Pre-Holdout Set\n(Label Blind,\nRandomly Selected)', **pre_node_style)
dot.node('D', 'Analysis Set', **anal_node_style)
dot.node('CV', 'Cross-Validation Set', **anal_node_style)
dot.node('Test', 'Institute Hold-Out', **anal_node_style)

with dot.subgraph() as s:
    s.attr(rank='same')
    s.node('Pre', 'Pre-Holdout Set\n(Label Blind,\nRandomly Selected)', **pre_node_style)
    s.edge('UC', 'Pre', **pre_edge_style)
    s.edge('UM', 'Pre', **pre_edge_style)
    s.edge('OSU', 'Pre', **pre_edge_style)
    s.edge('MUSC', 'Pre', **pre_edge_style)
    s.edge('MD', 'Pre', **pre_edge_style)
    s.edge('UL', 'Pre', **pre_edge_style)



institutions = ['UC', 'UM', 'OSU', 'MUSC', 'MD', 'UL']
for inst in institutions:
    if inst in ['UC', 'UM', 'OSU', 'MUSC']:
        dot.edge(inst, 'D', **anal_1_edge_style)
    else:
        dot.edge(inst, 'D', **anal_2_edge_style)

dot.edge('D', 'CV', **anal_1_edge_style)
dot.edge('D', 'Test', **anal_2_edge_style)

dot.view()

In [ ]:
from graphviz import Digraph

dot = Digraph(comment='Sample Processing Diagram')
dot.attr(rankdir='TB', nodesep='0.2', ranksep='0.4')

node_style = {'shape': 'rect', 'style': 'filled,rounded', 'fillcolor': '#AED6F1', 'fontname': 'Helvetica', 'fontsize': '10'}
anal_node_style = {'shape': 'rect', 'style': 'filled,rounded', 'fillcolor': 'khaki', 'fontname': 'Helvetica', 'fontsize': '10'}
pre_node_style = {'shape': 'rect', 'style': 'filled,rounded', 'fillcolor': 'lightsalmon', 'fontname': 'Helvetica', 'fontsize': '10'}
pre_edge_style = {'fontsize': '9', 'fontname': 'Helvetica', 'color': 'salmon'}

# Nodes
dot.node('UC', 'UC', **node_style)
dot.node('UM', 'U-M', **node_style)
dot.node('OSU', 'OSU', **node_style)
dot.node('MUSC', 'MUSC', **node_style)
dot.node('MD', 'MD Anderson', **node_style)
dot.node('UL', 'UofL', **node_style)

dot.node('Pre', 'Pre-Holdout Set\n(Label Blind,\nRandomly Selected)', **pre_node_style)
dot.node('D', 'Analysis Set', **anal_node_style)
dot.node('CV', 'Cross-Validation Set', **anal_node_style)
dot.node('Combo', 'Institute Hold-Out\n(All Combinations\nof 2 Institutes)', **anal_node_style)

# Horizontal grouping
with dot.subgraph() as s:
    s.attr(rank='same')
    s.node('Pre', 'Pre-Holdout Set\n(Label Blind,\nRandomly Selected)', **pre_node_style)
    for inst in ['UC', 'UM', 'OSU', 'MUSC', 'MD', 'UL']:
        s.edge(inst, 'Pre', **pre_edge_style)

# Simulated striped edges to Analysis Set
institutions = ['UC', 'UM', 'OSU', 'MUSC', 'MD', 'UL']
for inst in institutions:
    dot.edge(inst, 'D', color='mediumseagreen', penwidth='1.5')
    dot.edge(inst, 'D', color='orchid', penwidth='1.5', style='dashed', constraint='false')

# Downstream analysis
dot.edge('D', 'CV', color='mediumseagreen', penwidth='1.5')
dot.edge('D', 'Combo', color='orchid', penwidth='1.5', style='dashed')

# Render
dot.view()
